<a href="https://colab.research.google.com/github/EverHernandez01/Senales_y-_Sistemas_2025_S2/blob/main/Soluci%C3%B3n_Taller/Ejercicio_1_7_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

$$Ejercicio\ 1.7$$

Aplicacion en circuitos electricos - potencia. Consulte en
que consiste la distorsion total de armonicos (Total Harmonic Distortion-(THD)) y el factor de potencia en un circuito electrico. Como puede calcularse el THD desde la FFT?.Como puede calcularse la distorsion del factor de potencia con base al THD?. Genere un ejemplo ilustrativo para el calculo del THD y la distorsion del factor de potencia para un rectificador de onda completa con carga: i) netamente resistiva y ii) carga RC en serie. Establezca las condiciones necesarias para las simulaciones y pruebe con diferentes valores de R y C. Discuta los resultados obtenidos. Ver cuaderno Rectificador RC y THD.

En los sistemas eléctricos de corriente alterna (AC), lo ideal es que las tensiones y corrientes sean ondas sinusoidales puras de una sola frecuencia (la fundamental, por ejemplo 60 Hz).
Sin embargo, cuando conectamos cargas no lineales (como rectificadores, fuentes conmutadas, variadores, motores electrónicos, etc.), la forma de onda de corriente o tensión se distorsiona, apareciendo componentes de frecuencias múltiplos enteros de la fundamental, llamados armónicos.

Matemáticamente, cualquier señal periódica puede expresarse como suma de senos:

$$
i(t) = I_1 \sin(\omega t + \phi_1) + I_2 \sin(2\omega t + \phi_2) + I_3 \sin(3\omega t + \phi_3) + \cdots
$$

donde:

- $ I1 $: amplitud (o valor RMS) de la **componente fundamental** (frecuencia $ 1 \times f_0 $)  
- $ I2, I3, $: amplitudes RMS de las **componentes armónicas**



La **Distorsión Total de Armónicos (THD)** mide cuánta energía está contenida en las componentes armónicas respecto a la fundamental.  
Se define como:

$$
\text{THD} = \frac{\sqrt{I_2^2 + I_3^2 + I_4^2 + \cdots}}{I_1}
$$

También puede expresarse como:

$$
\text{THD} = \frac{\sqrt{I_{\text{total}}^2 - I_1^2}}{I_1}
$$

Y en porcentaje:

$$
\text{THD}(\%) = \frac{\sqrt{I_{\text{total}}^2 - I_1^2}}{I_1} \times 100
$$


El factor de potencia indica qué tan eficientemente se utiliza la potencia eléctrica suministrada.
Se define como la relación entre la potencia activa (P) y la potencia aparente (S):


$$
PF = \frac{P}{S}
$$




**Potencia activa (P):**  
Es la parte de la potencia que realiza trabajo útil, como generar **calor, movimiento o luz**.

**Potencia aparente (S):**  
Es el producto de los valores RMS de la tensión y la corriente totales:

$$
S = V_{\text{RMS}} \, I_{\text{RMS}}
$$



Si la corriente y la tensión son **sinusoidales** y están **desfasadas** por un ángulo $ \phi $, entonces el factor de potencia puede expresarse como:

$$
PF = \cos(\phi)
$$

donde $cos(\phi) $ se conoce como el **factor de desplazamiento** (*displacement factor*), el cual representa el desfase entre la corriente y la tensión fundamental.


Cuando hay distorsión (no solo desfase), el factor de potencia total no es solamente el cos(φ). Se descompone en dos factores:

$$
PF = \underbrace{\cos(\phi_1)}_{\text{Displacement factor}} \times \underbrace{\frac{I_1}{I_{\text{total}}}}_{\text{Distortion factor}}
$$

Relacion entre el THD y el FP

A partir de las definiciones:

$$
I_{\text{total}} = I_1 \sqrt{1 + THD^2}
$$

y sustituyendo en la ecuación del PF:

$$
PF = \frac{I_1}{I_{\text{total}}} \cos(\phi_1) = \frac{\cos(\phi_1)}{\sqrt{1 + THD^2}}
$$

Esto muestra que:

Si el **THD** aumenta, el **PF total** disminuye, incluso si el desfase $( \phi_1 $) es cero. Es decir, una forma de onda muy distorsionada puede tener bajo factor de potencia aunque sea “no reactiva”.


In [ ]:
#paquetes de para simulación
import numpy as np
import scipy
import matplotlib.pyplot as plt
#%matplotlib inline
import scipy.signal as sig
import scipy.optimize as opt
#from IPython.display import Image
import sympy as sym
#sym.init_session()

In [ ]:
s = sym.symbols('s', complex=True)
t, R, L, C = sym.symbols('t R L C', positive=True)
X = sym.Function('X')(s)

Y = 1/(R*C*s + 1) * X
Y

#Cambiamos valores de R y C para la simulacion

In [ ]:
R_v = 100 #valores de simulacion de R
C_v =  470e-6 # valores de simulacion de C
Y_RC = Y.subs(R, R_v).subs(C, C_v)
Y_RC

In [ ]:
num = np.array([1]) #coeficientes del numerador de la funcion de transferencia como numpy array
den = np.array([R_v*C_v,1])#coeficientes del denominador de la funcion de transferencia como numpy array
G_n = sig.TransferFunction(num, den) #funcion de transferencia en
print(num)
print(den)

In [ ]:
Fo = 60 #frec alimentación
Fs = 30*Fo #frecuencia muestreo
To = 1/Fo #periodo fundamental
Ts = 1/Fs #periodo muestreo
t = np.arange(0, 8*To,Ts) # se simulan 5 peridos de alimentación con un muestreo segun Ts
A = 120 #amplitud entrada
#función de entrada aproximada como onda senoidal rectificada completa
in_o = A*(np.sin(2*np.pi*Fo*t))
rec_c = sig.square(2*np.pi*Fo*t)#función tren de pulsos para simular señal rectificada
#Para rectificador de media onda:
rec_m = 0.5*(sig.square(2*np.pi*Fo*t)+1)
in_ = in_o * rec_c #si se quiere probar con rec media onda multiplicar por rec_m
out = G_n.output(in_, T=t)[1] #evaluar salida ante entrada rectificada mediante aproximacion numérica de scipy

#señales de alimentación y tren de pulsos
plt.plot(t,in_o,label='Alimentación')
plt.legend()
plt.show()

plt.plot(t,rec_c,label='Tren Pulsos')
plt.legend()
plt.show()

plt.plot(t, in_,label='In(t)')#entrada rectificada
plt.plot(t, out,label='Out(t)')#salida del circuito
plt.xlabel('t [s]')
plt.ylabel('out(t)')
plt.show()

#Calculo del espectro con fft

In [ ]:
vfre = np.fft.rfftfreq(len(out),1/Fs) #vector de frecuencia en Hz segun tamaño del vector y tiempo de muestreo
Xf = np.fft.rfft(out)

plt.stem(vfre,abs(Xf/len(out)))
plt.xlabel('Frecuencia [Hz]')
plt.ylabel('$|X(f)|$')
plt.show()